In [113]:
!pip install yfinance

In [169]:
import pandas as pd
import numpy as np
import tensorflow as tf
from pandas_datareader import data
import keras
import datetime as dt
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler
import urllib.request, json
import yfinance as yf
from tensorflow.python.keras.layers import Input, Dense
from tensorflow.python.keras.models import Sequential

In [195]:
ticker = "AVGO"
file_name = "%s_stock.csv"%(ticker)

if not os.path.exists(file_name):
    df = yf.download(ticker)
    
    df.to_csv(file_name)
    df = pd.read_csv(file_name)
    print("Data Saved To: %s"%file_name)
else:
    print("File already exists! Reading file now")
    df = pd.read_csv(file_name)


[*********************100%***********************]  1 of 1 completed
Data Saved To: AVGO_stock.csv


In [196]:
df.corr()['Close']

Open         0.999660
High         0.999807
Low          0.999818
Close        1.000000
Adj Close    0.998289
Volume       0.185895
Name: Close, dtype: float64

In [197]:
columns = ['Open', 'High', 'Low', 'Adj Close', 'Volume']
df.drop(columns=columns, inplace= True)

In [198]:
df1 = df.drop(columns=['Date'])
df1

,Close
0,16.180000
1,16.430000
2,15.970000
3,15.670000
4,16.000000
...,...
2798,365.920013
2799,359.730011
2800,351.790009
2801,361.589996


In [199]:
sequence_length = 100
features = df1.shape[1]

In [200]:
scaler=MinMaxScaler(feature_range=(0,1))
scaler = scaler.fit(df1)
df_2_list = scaler.transform(df1)
index_length = df_2_list.shape[0]

index_length

2803

In [201]:
df_2_list

array([[0.00468767],
       [0.00538112],
       [0.00410518],
       ...,
       [0.93559303],
       [0.96277595],
       [0.93503824]])

In [202]:
i = 0
input_sequence = []
output_sequence = []
for i in range(index_length - sequence_length):
    temp_input = df_2_list[i:i+sequence_length]
    temp_output = df_2_list[i+sequence_length][0]
    input_sequence.append([temp_input])
    output_sequence.append([temp_output])

In [203]:
X = np.reshape(input_sequence, (len(input_sequence), sequence_length, features))
y = np.array(output_sequence)

In [204]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Embedding, Dropout

model = Sequential()
model.add(LSTM(1024, input_shape = (X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(1024, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1]))
model.summary()
model.compile(loss = 'mean_squared_error', optimizer = 'adam')

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_28 (LSTM)               (None, 100, 1024)         4202496   
_________________________________________________________________
dropout_28 (Dropout)         (None, 100, 1024)         0         
_________________________________________________________________
lstm_29 (LSTM)               (None, 100, 1024)         8392704   
_________________________________________________________________
dropout_29 (Dropout)         (None, 100, 1024)         0         
_________________________________________________________________
lstm_30 (LSTM)               (None, 100, 512)          3147776   
_________________________________________________________________
dropout_30 (Dropout)         (None, 100, 512)          0         
_________________________________________________________________
lstm_31 (LSTM)               (None, 256)              

In [205]:
#earlystop = keras.callbacks.EarlyStopping(monitor = 'loss', min_delta = 0.00001, patience = 1, mode = 'min')
#callbacks = [earlystop]

In [220]:
model.fit(X,y, batch_size = 1024, epochs = 10)

Epoch 1/10
3/3 [==============================] - 7s 2s/step - loss: 0.0018
Epoch 2/10
3/3 [==============================] - 7s 2s/step - loss: 0.0021
Epoch 3/10
3/3 [==============================] - 7s 2s/step - loss: 0.0016
Epoch 4/10
3/3 [==============================] - 8s 3s/step - loss: 0.0018
Epoch 5/10
3/3 [==============================] - 8s 3s/step - loss: 0.0014
Epoch 6/10
3/3 [==============================] - 7s 2s/step - loss: 0.0014
Epoch 7/10
3/3 [==============================] - 7s 2s/step - loss: 0.0015
Epoch 8/10
3/3 [==============================] - 7s 2s/step - loss: 0.0015
Epoch 9/10
3/3 [==============================] - 7s 2s/step - loss: 0.0014
Epoch 10/10
3/3 [==============================] - 7s 2s/step - loss: 0.0014


In [221]:
test_list = df_2_list[index_length - sequence_length - 30:]
input_sequence_test = []
final_test = test_list[:sequence_length]
input_sequence_test.append([final_test])
LSTM_input = np.reshape(input_sequence_test, (len(input_sequence_test), sequence_length, features))
LSTM_input.shape

(1, 100, 1)

In [222]:
predictions = []

for i in range(30):
  LSTM_input = np.reshape(input_sequence_test, (len(input_sequence_test), sequence_length, features))
  price = model.predict(LSTM_input)
  input_sequence_test = []
  final_test = np.vstack([final_test, [price[0][0]]])
  final_test = final_test[1:]
  input_sequence_test.append([final_test])
  predictions.append(price[0])

In [223]:
inversed = scaler.inverse_transform(predictions)
print(inversed)

[[313.70815014]
 [314.71291724]
 [315.38379343]
 [315.72153081]
 [315.81410399]
 [315.75107773]
 [315.57512852]
 [315.29248808]
 [314.91381479]
 [314.47667084]
 [314.03213478]
 [313.61420169]
 [313.22007805]
 [312.82036735]
 [312.38767155]
 [311.92160385]
 [311.44973421]
 [311.00809912]
 [310.61614583]
 [310.2646557 ]
 [309.92433968]
 [309.56653188]
 [309.18106817]
 [308.77970285]
 [308.38658917]
 [308.02091652]
 [307.68625202]
 [307.37026117]
 [307.05427032]
 [306.72542925]]


In [224]:
actual_values = test_list[len(test_list) - 30:]
actual_values = scaler.inverse_transform(actual_values)
print(actual_values)

[[333.64001465]
 [329.42001343]
 [327.82000732]
 [330.32000732]
 [328.64001465]
 [329.20999146]
 [328.82998657]
 [330.01998901]
 [334.        ]
 [336.14001465]
 [339.67001343]
 [339.14001465]
 [344.82000732]
 [347.1499939 ]
 [359.54000854]
 [375.01000977]
 [352.08999634]
 [362.95001221]
 [350.54000854]
 [360.02999878]
 [357.07998657]
 [359.70001221]
 [362.20001221]
 [366.91000366]
 [367.04000854]
 [365.92001343]
 [359.73001099]
 [351.79000854]
 [361.58999634]
 [351.58999634]]


In [225]:
final_predict = test_list[len(test_list) - 100:]
input_sequence_predict = []
input_sequence_predict.append([final_predict])
LSTM_input_predict = np.reshape(input_sequence_predict, (len(input_sequence_predict), sequence_length, features))
LSTM_input_predict.shape

(1, 100, 1)

In [226]:
predictions_final = []

for i in range(30):
  LSTM_input_predict = np.reshape(input_sequence_predict, (len(input_sequence_predict), sequence_length, features))
  price_predict = model.predict(LSTM_input_predict)
  input_sequence_predict = []
  final_predict = np.vstack([final_predict, [price_predict[0][0]]])
  final_predict = final_predict[1:]
  input_sequence_predict.append([final_predict])
  predictions_final.append(price_predict[0])

inversed_predict = scaler.inverse_transform(predictions_final)
print(inversed_predict)

[[348.73977038]
 [349.0687189 ]
 [348.96858171]
 [348.40140983]
 [347.49920814]
 [346.47937748]
 [345.51965063]
 [344.67808996]
 [343.89725627]
 [343.07262866]
 [342.13402517]
 [341.08862301]
 [340.00868855]
 [338.98144431]
 [338.05854903]
 [337.23389994]
 [336.45727802]
 [335.66958944]
 [334.83754825]
 [333.9673217 ]
 [333.09492479]
 [332.26105706]
 [331.48714271]
 [330.76641282]
 [330.07110402]
 [329.37063793]
 [328.64930635]
 [327.91252442]
 [327.18137253]
 [326.4777691 ]]
